# Data Wrangling the OpenStreetMap: San Francisco

_An exercise in auditing, cleaning, and exploring the City of San Francisco's OpenStreetMap (OSM) data provided by the [OpenStreetMap Project](https://www.openstreetmap.org/relation/111968)._

**Ravi Dayabhai**

In [2]:
# Import dependencies
import xml.etree.cElementTree as ET
import pprint
import re
from collections import defaultdict
import sqlite3
import pandas as pd
import math
import folium
from pyzipcode import ZipCodeDatabase
import numpy as np
import os


# Initialize PrettyPrinter object
pp = pprint.PrettyPrinter()

## Introduction

I chose San Francisco as the map area to explore because I am relatively new to the area having lived the entirety of my adult life on the east coast. I figured using the following map extract to complete this project would also give me a better sense of my new home city!

>**NOTE**: _Cleaning_, for the purposes of this project, simply means that various tags were standardized (according to convention of my choosing) before being exported to tables (and, subsequently, read into a SQLite database). More on this later in this project document.

**Map Area**
- [San Francisco](https://www.openstreetmap.org/relation/111968); custom extract for the _City of San Francisco_ rather than the entire bay area or outlying islands.

**Files**
- `audit.py`
 - This script houses the functions used by `data.py` to clean the OSM file.
- `data.py`
 - This is the script (skeleton provided by the Case Study quiz) that parsed the OSM file and generated the CSV flatfiles according to the schema outlined in `schema.sql` (i.e., the prescribed schema provided by Udacity)
- `database.py`
 - This is a script I wrote to load all of the CSVs generated by `data.py` into a SQLite database according to the schema outlined in `schema.sql`.
- `SF_OSM.db`
    - The SQLite database housing the _cleaned_ information and which I query later on in the project.
      - **NOTE**: This repo houses only a _sample_ of the map extract (due to file size limits and submission requirements). 
- `schema.sql`
 - The schema suggested by Udacity for the SQLite database.


Here's a quick preview of the particular map area in question:

![City of San Francisco](map_extract_screenshot.png "SF Map Extract")

### Overview

#### Size of Database

The final, cleaned database size (vs. original OSM file) is calculated below:

In [3]:
database_size = os.path.getsize('./SF_OSM.db') / 1000000
osm_size = os.path.getsize('./SF_extract.osm') / 1000000

print("The size of the database is: %s MB" %round(database_size, 2))
print("The size of the OSM file is: %s MB" %round(osm_size, 2))

The size of the database is: 189.0 MB
The size of the OSM file is: 359.0 MB


#### Tables

The following command (`ANALYZE`) creates a summary of the table of the database, which I'm outputting to a Pandas DataFrame for ease of viewing.

In [4]:
# Set up database environment
conn = sqlite3.connect('SF_OSM.db')
c = conn.cursor()

for row in c.execute('ANALYZE;'):
    print row
    
stat_query = '''
SELECT tbl AS "Table", stat AS "Total Rows" FROM sqlite_stat1;
'''

# Query to Pandas DataFrame
df_db_stats = pd.read_sql_query(stat_query, conn)
df_db_stats

,Table,Total Rows
0,ways_nodes,2103139
1,ways_tags,479392
2,ways,177567
3,nodes_tags,114601
4,nodes,1681560


This isn't the last time you'll see the database API interface! More to come on that below in the following sections:
 - _Confirming Cleaned Data_
 - _Exploring the Data_

## Auditing

First, I took a look at the various kinds of `tag` tags present in the OSM file.

**Observation #1**: Here we see that `addr:source:housenumber`, `addr.source:housenumber`, `addr:housenumber:source` are all inconsistent ways of conveying the same information. There is also some inconsistency in `tag` key conventions around "alternate names" (e.g., `alt_name_1` vs. `alt_name2`)

>**Note**: I chose to ignore `addr.source:housenumber` in the `shape_element` function, but standardizing tags is definitely a point of potential improvement!

**Observation #2**: Also, there are some very odd `way` tags (e.g., `Stevenson Place`, [`FIXME`](http://wiki.openstreetmap.org/wiki/Key:fixme), etc.) some of which reflect the ever-evolving nature of the map.

For the purposes of this project, I have decided to "clean" the following tags with the following conventions in mind:
1. Streets
 - _Convention_: e.g., `Waller Street`
 - This convention favors street names without specifying numbers (i.e., without suite number, house number, unit number) or abbreviations _unless_ the street has a widely known and accepted name that deviates from the aforementioned convention. 
2. Phone Numbers
 - _Convention_: e.g., `(415) 123-4567`
3. Zip Codes
 - _Convention_: e.g., `94102`

### Streets

The first attribute I will clean will be `addr:street` according to the conventions specified above. The code below checks the _street type_ by comparing the last word in a street name to a list of expected street types (as per the Udacity case study preceding this project).

In [7]:
# Audits elements that have 'addr:street' tags
def street_audit():

    # Initialize variables and Regex and helper functions
    street_types = defaultdict(set)
    street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

    expected_streets = ["Avenue", "Street", "Boulevard", "Drive", "Lane", "Place",
                "Road", "Circle", "Parkway", "Court", "Way", "Terrace", "Alley",
                "Crescent", "Highway", "Junction", "Loop", "Trail", "Landing", "Walk",
                "Route", "Center", "Plaza"]

    def audit_street_type(street_types, street_name):
        m = street_type_re.search(street_name)
        if m:
            street_type = m.group()
            if street_type not in expected_streets:
                street_types[street_type].add(street_name)

    def is_street_name(elem):
        return (elem.attrib["k"] == "addr:street")
    
    # Loop through all top level tags
    for event, elem in ET.iterparse(osm_data, events=("start",)):
        for tag in elem.iter("tag"):

            # Audits street names
            if is_street_name(tag):
                audit_street_type(street_types, tag.attrib["v"])
                    
    return street_types

with open("SF_extract.osm", "r") as osm_data:
    unexpected_streets = street_audit()

unexpected_streets

defaultdict(set,
            {'151': {'Brannan St #151'},
             '15th': {'15th'},
             '2': {'San Francisco Bicycle Route 2'},
             '203': {'Bartlett Street #203'},
             '3500': {'Sansome St #3500'},
             '3658': {'Market Street Suite 3658'},
             '39': {'Pier 39'},
             '4.5': {'SF 80 PM 4.5'},
             '404': {'16th St #404'},
             '730': {'Sansome Street Ste 730', 'Sansome Street Suite 730'},
             'A': {'Avenue A', 'Pier 50 A'},
             'Ave': {'Geneva Ave',
              'Grant Ave',
              'Jerrold Ave',
              'Morena Ave',
              'Pennsylvania Ave',
              'Phelan Ave',
              'Van Ness Ave'},
             'B': {'Avenue B', 'Pier 50 B'},
             'Bldg': {'Ferry Bldg'},
             'Blvd': {'Marina Blvd', 'Terry A Francois Blvd'},
             'Broadway': {'Broadway'},
             'Building': {'Ferry Building', 'Multi Use Building'},
             'California':

**Observation #3**: Here we see that there are inconsistencies with street information, which is unsurprising given the variable nature people refer to streets. 

Here are some common issues, as noted in the audit above (and subsequent audit iterations):

- Variable street type abbreviations (e.g, "Ave" vs. "AVE" vs. "Ave.")
- Sequencing of street modifiers (e.g., "North Willard Street" vs. "Willard North")
- Many streets were named correctly, but lacked a proper suffix (e.g., "Hyde" vs. "Hyde Street")
- Some streets were listed as intersections and not detected by the above approach (e.g., "Murray Street And Justin Drive" would pass the naïve regex in the audit function above).

The second street audit below checks for streets that have numbers either prepended or appended for whatever reason (e.g., house numbers).

In [8]:
# Audits elements that have 'addr:street' tags
def street_audit2():

    # Initialize variables and Regex and helper functions
    streets_with_prefixes = []
    units_re = re.compile(r'^#?\d+\b.*|.*\b#?\d+$')

    def audit_street_prefix(street_name):
        m = units_re.search(street_name)
        if m:
            streets_with_prefixes.append(m.group())

    def is_street_name(elem):
        return (elem.attrib["k"] == "addr:street")
    
    # Loop through all top level tags
    for event, elem in ET.iterparse(osm_data, events=("start",)):
        for tag in elem.iter("tag"):

            # Audits street names
            if is_street_name(tag):
                audit_street_prefix(tag.attrib["v"])
                    
    return streets_with_prefixes

with open("SF_extract.osm", "r") as osm_data:
    unexpected_streets2 = street_audit2()

test_cases = set(unexpected_streets2)
test_cases

{'16th St #404',
 '2640 mason st',
 '30 Rickard Street',
 '303 Second Street',
 '434 Main Street',
 'Bartlett Street #203',
 'Brannan St #151',
 'Market Street Suite 3658',
 'Pier 39',
 'SF 80 PM 4.5',
 'San Francisco Bicycle Route 2',
 'Sansome St #3500',
 'Sansome Street Ste 730',
 'Sansome Street Suite 730'}

(See `audit.py` for the functions defined specifically to fix these issues raised in the auditing procedure above!)

### Phone Numbers


I chose the `phone` attribute (read: phone numbers) because that's another feature that has decent conventions by which we can standardize the available data.

In [9]:
# Initialize variables and Regex and helper functions
phone_re = re.compile(r'''(
    (\d{3}|\(\d{3}\))?                # area code
    (\s|-|\.)?                        # separator
    (\d{3})                           # first 3 digits
    (\s|-|\.)?                         # separator
    (\d{4})                           # last 4 digits
    (\s*(ext|x|ext.)\s*(\d{2,5}))?    # extension
    )''', re.VERBOSE)

# Audits elements that have 'phone' tags
def phone_audit():

    thrown_out = set()
    weirdest_nums = set()
    semi_normal_nums = set()
    
    def audit_phone_num(phone_num):

        # Checks to see if number characters present is conceivably a phone number
        stripped_num = re.sub(r'[^0-9a-zA-Z]', '', phone_num)
        if len(stripped_num) < 7 or len(stripped_num) > 15:
            thrown_out.add(phone_num)
            return
        
        m = phone_re.search(phone_num)
        if not m:
            weirdest_nums.add(phone_num)
        else:
            semi_normal_nums.add(phone_num)

    def is_phone_number(elem):
        return (elem.attrib["k"] == "phone")
    
    # Loop through all top level tags
    for event, elem in ET.iterparse(osm_data, events=("start",)):
        for tag in elem.iter("tag"):

            # Audits phone numbers
            if is_phone_number(tag):
                audit_phone_num(tag.attrib["v"])
            
    return weirdest_nums, semi_normal_nums, thrown_out

with open("SF_extract.osm", "r") as osm_data:
    weird_nums, semi_norm_nums, thrown_out = phone_audit()

pp.pprint(weird_nums)

set(['(+1) 415 291 20 33',
     u'(415) 549\u20137932',
     '(415) 735 - 6843',
     '+1 415 759 TOYS',
     '415 242 960',
     '415 409 FARM',
     '415-397-BROS'])


**Observation #4**: The mnemonic endings (e.g., 'FARM', 'BROS') to a few of these phone required a little bit of decoding, otherwise most of the phone numbers follow common phone number formats (given the dearth of items in `weird_nums`).

>**Note**: The regular expression (regex) employed in the phone number audit function above captured most of the variability in phone number formats.

(See `audit.py` for defined functions that deal with these inconsistencies among formats!)

### ZIP Codes


The code below separates the _obviously_ suspect (read: `weird`) zip (read: postal) codes from those that _appear_ normal. The 

In [10]:
# Initialize variables and Regex and helper functions
zip_re = re.compile(r'^(94\d{3})(-\d{4})?$')

# Audits elements that have 'postcode' tags
def zip_audit():
   
    weird_zips = set()
    normal_zips = set()
    
    def audit_zip(zip_code):
        m = zip_re.search(zip_code)
        if not m:
            weird_zips.add(zip_code)
        else:
            normal_zips.add(m.group(1))

    def is_zip(elem):
        return (elem.attrib["k"] == "addr:postcode")
    
    # Loop through all top level tags
    for event, elem in ET.iterparse(osm_data, events=("start",)):
        for tag in elem.iter("tag"):

            # Audits zip numbers
            if is_zip(tag):
                audit_zip(tag.attrib["v"])
            
    return weird_zips, normal_zips

with open("SF_extract.osm", "r") as osm_data:
    weird, normal = zip_audit()

pp.pprint(weird)

set(['14123', '41907', '90214', '95115', 'CA'])


**Observation #5**: Zip codes were generally already pretty clean (or followed standard conventions, even if these conventions were not aligned with the one with which I'm subscribing to).

These `weird` zip codes are corrected in `audit.py`, which demonstrates that many errors likely resulted from user input, such as:
- Switched digits (e.g., `94013` vs. `94103`)
- Incorrectly substituted digits (e.g., `14123` vs. `94123`)
- Incorrect digits repeated (e.g., `94113` vs. `94133`)

One way the above error patterns were made evident to me was by employing the code below, which returns a table consisting of zip codes that appear normal (i.e., their first two digits are `94`) but do not map to cities in the map extract area (i.e., San Francisco, Daly City, and Sausalito).

In [11]:
# Custom helper function to check each zipcode against database
def zip_city_checker(z):
    zcdb = ZipCodeDatabase()
    try:
        return zcdb[z].city
    except:
        return np.nan

# Checks the seemingly 'normal' zipcodes for issues
cities = []
for zipcode in normal:
    cities.append(zip_city_checker(zipcode))

df_zip_cities = pd.DataFrame({'ZIP': list(normal), 'City': cities})
ok_cities = ['San Francisco', 'Daly City', 'Sausalito']
cond = df_zip_cities['City'].apply(lambda x: x not in ok_cities)
df_zip_cities[cond]

,City,ZIP
8,NaN,94113
13,NaN,94158
17,Sunnyvale,94087
35,NaN,94013


In my cleaning code (`audit.py`) I revisited the `94...` zip codes in the above table, which required me to find the associated `nodes` in the OSM file and to replace these erroneous zip codes with the actual zip code contained in the researched address for given `node`s.

## Confirming Cleaned Data

Here, I'm checking **streets** by looking at the most infrequently appearing streets as those are the most likely to be aberrant.

In [12]:
# Query
final_query = '''
SELECT tags.value, COUNT(*) AS num, tags.type
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags
WHERE (tags.key == "street") AND (tags.type == "addr")
GROUP BY tags.value
HAVING (num < 3)
ORDER BY num, tags.value;
'''


# Query to Pandas DataFrame
df_streets = pd.read_sql_query(final_query, conn)
df_streets

,value,num,type
0,12th Street,1,addr
1,Amador Street,1,addr
2,Amber Drive,1,addr
3,Aptos Avenue,1,addr
4,Arlington Street,1,addr
5,Armory Drive,1,addr
6,Avenue B,1,addr
7,Avenue E,1,addr
8,Avenue F,1,addr
9,Avenue G,1,addr


Looks good. Continuing onto checking **zip codes**:

In [13]:
# Query
final_query = '''
SELECT tags.value, COUNT(*) AS num, tags.type
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags
WHERE (tags.key == "postcode")
GROUP BY tags.value
ORDER BY num, tags.value;
'''


# Query to Pandas DataFrame
df_zips = pd.read_sql_query(final_query, conn)
df_zips

,value,num,type
0,94015,1,addr
1,94017,1,addr
2,94164,1,addr
3,94188,1,addr
4,94130,3,addr
5,94166,4,addr
6,94143,6,addr
7,94134,8,addr
8,94965,9,addr
9,94129,10,addr


Here are the zipcodes that are _apparently not_ in the map extract area, even after cleaning has been completed.

In [14]:
df_zips['cities'] = df_zips['value'].apply(zip_city_checker)
cities = ['San Francisco', 'Daly City', 'Sausalito']
m = df_zips['cities'].apply(lambda x: x not in cities)
df_zips[m]

,value,num,type,cities
10,94158,29,addr,NaN


`94158` is, indeed, a San Francisco zip code (the 29 instances are no coindence), but was not a zip code when the external Python library was written (early-mid 2000s)!

Finally, I checked to see if there were any remaining problematic **phone** data:

In [15]:
# Query
final_query = '''
SELECT tags.id, tags.value, tags.type
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags
WHERE (tags.key == "phone") AND (tags.value IS NULL)
ORDER BY tags.value DESC;
'''


# Query to Pandas DataFrame
df_phones = pd.read_sql_query(final_query, conn)
df_phones

,tags.id,tags.value,tags.type


(This empty table means everything has been cleaned!)

## Exploring the Data
First, let's take a look at some of the standard queries as per the project rubric, chiefly:
- number of unique users/contributors
- number of nodes and ways

### Contributors
This query takes a look at the top 10 contributors to the OSM SF extract.

In [16]:
# Query
final_query = '''
SELECT x.user, COUNT(*) AS "Number of Tags"
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) x
GROUP BY x.user
ORDER BY "Number of Tags" DESC
LIMIT 10;
'''


# Query to Pandas DataFrame
df_top_users = pd.read_sql_query(final_query, conn)
df_top_users

,user,Number of Tags
0,ediyes,645049
1,Luis36995,452474
2,Rub21,328103
3,KindredCoda,55016
4,AndrewSnow,40220
5,upendra_sfimport,27238
6,saikabhi_sfimport,26579
7,osmmaker,21056
8,BharataHS_sfimport,20785
9,brentengust,19895


### Nodes and Ways

Counting nodes and ways requires understanding that OSM `id`s are **_not_** unique across object type (i.e., `node`, `way`, etc.), but _are_ unique within an object type. This distinction is illustrated below (and corroborated [here](https://gis.stackexchange.com/questions/103572/are-osm-ids-unique-over-all-object-types)).

In [17]:
# Query
final_query = '''
SELECT COUNT(id), COUNT(DISTINCT id)
FROM (SELECT id,user,uid,version,changeset,timestamp FROM nodes UNION ALL SELECT * FROM ways) x
LIMIT 10;
'''


# Query to Pandas DataFrame
df_ways_nodes = pd.read_sql_query(final_query, conn)
df_ways_nodes

,COUNT(id),COUNT(DISTINCT id)
0,1859127,1859095


Therefore, the best **count of `way`s and `node`s** in the database is **1,859,127**.

### Cafes

The first exercise is **finding all of the coffee shops with internet access and all coffee shops around my home**. The second is to **find all of the coffee shops around my home that have internet access**.

Here are the cafes with internet access in San Fransisco:

In [18]:
# Query for cafes with internet access
query_cafe_names = 'SELECT nodes_tags.id, nodes.lat, nodes.lon, nodes_tags.value FROM nodes_tags LEFT JOIN nodes ON nodes_tags.id == nodes.id WHERE key == "name"'
query_cafe_amenities = 'SELECT nodes_tags.id, nodes_tags.key, nodes_tags.value FROM nodes_tags LEFT JOIN nodes ON nodes_tags.id == nodes.id WHERE key == "amenity"'
query_cafe_internetaccess = 'SELECT nodes_tags.id, nodes_tags.key, nodes_tags.value FROM nodes_tags LEFT JOIN nodes ON nodes_tags.id == nodes.id WHERE key == "internet_access"'

query_cafes = 'SELECT DISTINCT q1.id AS "id", q1.lat AS "Latitude", q1.lon AS "Longitude", q1.value AS "Cafe Name", q3.value AS "Internet Access" FROM (({}) q1 JOIN ({}) q2 ON q1.id == q2.id) JOIN ({}) q3 ON q3.id == q1.id WHERE q2.value == "cafe";'.format(query_cafe_names, query_cafe_amenities, query_cafe_internetaccess)


# Query to Pandas DataFrame
df_internetcafe = pd.read_sql_query(query_cafes, conn)
df_internetcafe

,id,Latitude,Longitude,Cafe Name,Internet Access
0,331389010,37.798631,-122.407352,Caffe Trieste,yes
1,332787308,37.787094,-122.456387,Picnix Bistro,no
2,360450263,37.751121,-122.438327,Philz Coffee,wlan
3,376235927,37.806556,-122.420377,Starbucks,wlan
4,410845445,37.790168,-122.415411,Another Cafe,yes
5,443372068,37.756417,-122.421229,Ritual Coffee Roasters,yes
6,586713622,37.764849,-122.432012,Peet's Coffee & Tea,yes
7,679010814,37.775598,-122.393369,Philz Coffee,wlan
8,725100826,37.787163,-122.410788,Starbucks,wlan
9,747853234,37.796800,-122.421770,Crave,yes


Here are the coffee shops around my home:

In [19]:
# Do geographic calculations
'''
Each 1 degree of LAT is approximately 69.172 miles
Each 1 degree LONG is a function of LAT
'''

lat_home = 37.771715
long_home = -122.424494
delta_LAT_1degree_miles_EQUATOR = 69.172

delta_LAT_1mile_degrees = 1 / delta_LAT_1degree_miles_EQUATOR
delta_LONG_1degree_miles = math.cos(math.radians(lat_home)) * delta_LAT_1degree_miles_EQUATOR
delta_LONG_1mile_degrees = 1 / delta_LONG_1degree_miles

sq_miles = 1.0
delta_long = delta_LONG_1mile_degrees * sq_miles**0.5 / 2
delta_lat = delta_LAT_1mile_degrees * sq_miles**0.5 / 2

range_long = (long_home - delta_long, long_home + delta_long)
range_lat = (lat_home - delta_lat, lat_home + delta_lat)

# Query for cafes near me
query_cafe_names_nearme = 'SELECT nodes_tags.id, nodes.lat, nodes.lon, nodes_tags.value FROM nodes_tags LEFT JOIN nodes ON nodes_tags.id == nodes.id WHERE key == "name"'
query_cafe_amenities_nearme = 'SELECT nodes_tags.id, nodes_tags.key, nodes_tags.value FROM nodes_tags LEFT JOIN nodes ON nodes_tags.id == nodes.id WHERE key == "amenity" AND value == "cafe" AND lat >= {} AND lat <= {} AND lon >= {} AND lon <= {} ORDER BY value'.format(range_lat[0], range_lat[1], range_long[0], range_long[1])
query_cafes_nearme = 'SELECT DISTINCT q1.id AS "id", q1.lat AS "Latitude", q1.lon AS "Longitude", q1.value AS "Cafe Name" FROM ({}) q1 JOIN ({}) q2 ON q1.id == q2.id WHERE q2.value == "cafe"'.format(query_cafe_names_nearme, query_cafe_amenities_nearme)
query_cafes_streets = 'SELECT nodes_tags.id, nodes.lat, nodes.lon, nodes_tags.value FROM nodes_tags LEFT JOIN nodes ON nodes_tags.id == nodes.id WHERE key == "street"'

final_query = 'SELECT cafes.*, streets.value AS "Street" FROM ({}) cafes LEFT JOIN ({}) streets ON cafes.id = streets.id'.format(query_cafes_nearme, query_cafes_streets)

# Query to Pandas DataFrame
df_nearme = pd.read_sql_query(final_query, conn)
df_nearme

,id,Latitude,Longitude,Cafe Name,Street
0,411083201,37.767316,-122.421948,Four Barrel Coffee,Valencia Street
1,418513660,37.764500,-122.421681,Muddy Waters,Valencia Street
2,528057543,37.768212,-122.422386,Carlin's Cafe,Valencia Street
3,586713622,37.764849,-122.432012,Peet's Coffee & Tea,Market Street
4,647656728,37.771672,-122.433137,Nectar,Haight Street
5,647656785,37.772169,-122.430725,Cafe International,Haight Street
6,1493456455,37.776359,-122.418088,Ma'velous,Market Street
7,1493456487,37.777046,-122.417274,Starbucks,Market Street
8,2005894602,37.769072,-122.427724,Starbucks,Market Street
9,2095344770,37.766319,-122.417422,Flying Pig Bistro,South Van Ness Avenue


Let's visually double-check these selected longitude & latitude value pairs on a map.

In [20]:
# Imports dependency used to drop markers onto a map
import folium

# Isolating "northing" and "easting" information with labels for each point
locationlist = df_nearme[["Latitude","Longitude"]].values.tolist()
labels = df_nearme["Cafe Name"].values.tolist()

# Create map and drop points onto it
m = folium.Map(location=[lat_home, long_home], zoom_start=14)
for point in range(len(locationlist)):
    folium.Marker(locationlist[point]).add_to(m)

m

Here are the coffee shops around my home with internet access (I'm using `pandas` because I'm already working with two DataFrames.

In [21]:
df = df_internetcafe.merge(df_nearme, how="inner", on=[u'id', u'Latitude', u'Longitude', u'Cafe Name'])
df[df["Internet Access"] != "no"]

,id,Latitude,Longitude,Cafe Name,Internet Access,Street
0,586713622,37.764849,-122.432012,Peet's Coffee & Tea,yes,Market Street
1,2005894602,37.769072,-122.427724,Starbucks,wlan,Market Street
2,2095344770,37.766319,-122.417422,Flying Pig Bistro,wlan,South Van Ness Avenue
3,2187987262,37.766425,-122.429039,Church Street Cafe,yes,None


**Observation #6**: Some of these data are outdated (e.g., Mission Creek Cafe and Little Bird Cafe are no longer in business). Additionally, I know there are a ton more around me (especially in Hayes Valley and the Mission neighborhoods) that aren't reflected in the data!

The next step is to take this observation to the next level: **find the coffee shops with internet along a bike route!**

I start with identifying `cycleways` ways tags, figuring out what nodes on on `cycleways`, and finally, joining those nodes with the nodes identified as cafes.

In [22]:
# Queries to find coffee shops on bike paths
cycleway_ids = '''
SELECT ways.id AS ways_id
FROM ways_tags LEFT JOIN ways ON ways_tags.id == ways.id 
WHERE key == "cycleway" 
OR (key == "bicycle" AND value == "yes")
OR (key == "highway" AND value == "cycleway")
'''

nodes_on_cycleways = 'SELECT cycleway_ids.ways_id AS cycleway_id, ways_nodes.node_id AS nodes_on_cycleway_id FROM ({}) cycleway_ids LEFT JOIN ways_nodes ON ways_nodes.id == cycleway_ids.ways_id ORDER BY cycleway_id, nodes_on_cycleway_id'.format(cycleway_ids.strip('\n'))
queryA = 'SELECT nodes_on_cycleway.*, nodes.lat, nodes.lon FROM ({}) nodes_on_cycleway LEFT JOIN nodes ON nodes.id == nodes_on_cycleway.nodes_on_cycleway_id'.format(nodes_on_cycleways)
queryB = 'SELECT qA.*, nodes_tags.key, nodes_tags.value FROM ({}) qA LEFT JOIN nodes_tags ON qA.nodes_on_cycleway_id == nodes_tags.id WHERE (nodes_tags.key == "amenity") AND (nodes_tags.value == "cafe")'.format(queryA)

# Query to Pandas DataFrame
df = pd.read_sql_query(queryB, conn)
df

,cycleway_id,nodes_on_cycleway_id,lat,lon,nodes_tags.key,nodes_tags.value


**Observation #7:** This doesn't seem to work because the `ways_nodes` are those designations that _define the path_ itself (rather than things _along_ the path).

In [23]:
# Query to see what types of `nodes` comprise bike paths
queryC = 'SELECT qA.*, nodes_tags.key, nodes_tags.value FROM ({}) qA LEFT JOIN nodes_tags ON qA.nodes_on_cycleway_id == nodes_tags.id WHERE (nodes_tags.key != "None") ORDER BY nodes_tags.key, nodes_tags.value'.format(queryA)

# Query to Pandas DataFrame
df = pd.read_sql_query(queryC, conn)
df

,cycleway_id,nodes_on_cycleway_id,lat,lon,nodes_tags.key,nodes_tags.value
0,26378110,2399952061,37.743991,-122.421991,access,motor_vehicle
1,359385451,667197531,37.832255,-122.480953,access,yes
2,359385451,667197531,37.832255,-122.480953,access,yes
3,368033990,667115758,37.832340,-122.480608,access,yes
4,368049578,667197531,37.832255,-122.480953,access,yes
5,368049578,667197531,37.832255,-122.480953,access,yes
6,419755787,667115758,37.832340,-122.480608,access,yes
7,24354980,1142601042,37.809409,-122.412610,amenity,ferry_terminal
8,31934807,1142601108,37.795122,-122.391557,amenity,ferry_terminal
9,46274020,590986805,37.778163,-122.387559,amenity,ferry_terminal


Well, that was wholly unsatisfying.

Below is a second method to get at the coffee shops on streets that have been designated as bike paths.

This probably isn't the _best_ way to achieve the objective of isolating cafes accessible by bike routes since a given street may have some stretches with bike lanes and other parts without.

In [24]:
# Queries to find coffee shops on bike paths
cycleways_info = 'SELECT DISTINCT cycleway_ids.*, value AS "Street" FROM ({}) cycleway_ids LEFT JOIN ways_tags ON ways_tags.id == cycleway_ids.ways_id WHERE key == "name"'.format(cycleway_ids.strip('\n'))
cafes_info = 'SELECT cafes.[Cafe Name], streets.value AS "Street" FROM (SELECT DISTINCT q1.id AS "id", q1.lat AS "Latitude", q1.lon AS "Longitude", q1.value AS "Cafe Name" FROM (SELECT nodes_tags.id, nodes.lat, nodes.lon, nodes_tags.value FROM nodes_tags LEFT JOIN nodes ON nodes_tags.id == nodes.id WHERE key == "name") q1 JOIN (SELECT nodes_tags.id, nodes_tags.key, nodes_tags.value FROM nodes_tags LEFT JOIN nodes ON nodes_tags.id == nodes.id WHERE key == "amenity" AND value == "cafe" ORDER BY value) q2 ON q1.id == q2.id WHERE q2.value == "cafe") cafes LEFT JOIN (SELECT nodes_tags.id, nodes.lat, nodes.lon, nodes_tags.value FROM nodes_tags LEFT JOIN nodes ON nodes_tags.id == nodes.id WHERE key == "street") streets ON cafes.id = streets.id'
final_query = 'SELECT cycleways_info.*, cafes.[Cafe Name] FROM ({}) cycleways_info LEFT JOIN ({}) cafes ON cycleways_info.Street == cafes.Street WHERE cafes.[Cafe Name] NOT NULL'.format(cycleways_info, cafes_info)

# Query to Pandas DataFrame
df = pd.read_sql_query(final_query, conn)
df

,ways_id,Street,Cafe Name
0,8915796,Fulton Street,Starbucks
1,8916191,Mariposa Street,Starbucks
2,8916792,7th Street,Sightglass Coffee
3,8916922,Howard Street,Gaslamp Cafe
4,8916922,Howard Street,Samovar Tea Lounge
5,8916922,Howard Street,Tony Bolony
6,8917177,Sanchez Street,Duboce Park Cafe
7,8917284,Post Street,Capital One 360 Café
8,8917325,Monterey Boulevard,Monterey Deli
9,8917325,Monterey Boulevard,Railroad Expresso Café


Unfortunately, because I can't join on a _unique_ key, solving for internet access from this list is not feasibly reliably (i.e., I can only join on `Cafe Name`, which presumes a level of uniqueness among the names of cafes).

## Suggestions (Improvement, Additional Analysis)

Given the observations above, I think there are several potential improvement opportunities:
- The schema above doesn't support look-ups of `nodes` via `ways_nodes` because the `ways_nodes` are often used to define the `way` itself.
 - Part of this is how the data are structured and interrelated: this is possible through the use of relation tags that could create _route_ overlays. The schema employed in this project doesn't capture `relations` which precludes this type of analysis.
   - This method _benefits_ cross-querying by route and more tightly integrate the relationship between `ways` and `nodes`.
   - A _problem_ of this approach is that this would require more data upkeep as creating the routes requires a new relationship. Furthermore, uniqueness of tag `id`s would need to be universal rather than limited just to object type since you conceivably have a `ways_node` (or `node` on a route) and a `way` that share the same `id`.
- While the OSM is a living, breathing document subject to change by any user, it does suffer from frictions in capturing data, especially given the ever-evolving nature of the underlying city! While the above exploration of cafés highlights one such aspect, I'm certain there are others.
 - I wonder if integrations with other, _more popular_ geolocation-based apps (or apps that support geo-tagging) could help "fill the funnel" of data.
   - The obvious _benefit_ to this would be to reduce the friction in capturing more up-to-date information since 
   - An anticipated _problem_ might be getting these apps to share this data in the first place. Additionally, without a translation layer (to map to the data types present in the extant OSM Project, this data could muddle the decent data structure that already exists.
- While the OSM Project doesn't restrict tag types, I think that the redundancies observed (e.g., multiple ways of tagging a house number) above might be reduced if tag types for the _most commonly_ used tags were standardized more (or across geographies in a given country).
 - The 